In [23]:
# -*- coding: utf-8 -*-
"""UAS_Penalaran_Komputer_Tahap_3.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1A_YOUR_NOTEBOOK_ID_HERE
"""

# Instalasi Library
print("Menginstal library yang dibutuhkan untuk Tahap 3...")
# scikit-learn untuk TF-IDF dan SVM/Naive Bayes
# transformers untuk BERT
# sentence-transformers untuk kemudahan embedding jika menggunakan BERT (opsional)
# joblib untuk menyimpan/memuat model dan vektor
!pip install pandas scikit-learn transformers sentence-transformers joblib > /dev/null 2>&1
print("Instalasi library selesai.")

# Koneksi Google Drive
print("Menghubungkan Google Drive...")
from google.colab import drive
drive.mount('/content/drive')
print("Google Drive terhubung.")

Menginstal library yang dibutuhkan untuk Tahap 3...
Instalasi library selesai.
Menghubungkan Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive terhubung.


In [17]:
import pandas as pd
import os
import re # Diperlukan untuk clean_text
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.svm import SVC # Support Vector Machine
from sklearn.naive_bayes import MultinomialNB # Naive Bayes
from transformers import AutoTokenizer, AutoModel # Untuk BERT
from sentence_transformers import SentenceTransformer # Untuk BERT embedding yang lebih mudah
import torch # Untuk tensor di PyTorch (digunakan oleh transformers)
import json # Untuk menyimpan queries.json
import numpy as np # Untuk operasi numerik
import scipy.sparse # <<< TAMBAH BARIS INI
from typing import Dict, Any, List, Union, Tuple # Untuk type hinting

# --- Fungsi untuk membuat jalur (sama seperti Tahap 1 dan 2) ---
def create_path(folder_name: str) -> str:
    path = os.path.join('/content/drive/MyDrive/', folder_name)
    if not os.path.exists(path):
        os.makedirs(path)
        print(f"Folder '{path}' berhasil dibuat di Google Drive.")
    else:
        print(f"Folder '{path}' sudah ada di Google Drive.")
    return path

# --- Fungsi clean_text (DIBAWAH DARI TAHAP 1/2) ---
def clean_text(text: str) -> str:
    """
    Membersihkan teks putusan dari header, footer, disclaimer, dan normalisasi spasi.
    """
    if not isinstance(text, str):
        return ""

    text = re.sub(r'M a h ka m a h A g u n g R e p u blik In d o n esia\n', '', text)
    text = re.sub(r'Disclaimer\n', '', text)
    text = re.sub(r'Kepaniteraan Mahkamah Agung Republik Indonesia berusaha untuk selalu mencantumkan informasi paling kini dan akurat sebagai bentuk komitmen Mahkamah Agung untuk pelayanan publik, transparansi dan akuntabilitas\n', '', text)
    text = re.sub(r'pelaksanaan fungsi peradilan\. Namun dalam hal-hal tertentu masih dimungkinkan terjadi permasalahan teknis terkait dengan akurasi dan keterkinian informasi yang kami sajikan, hal mana akan terus kami perbaiki dari waktu kewaktu\.\n', '', text)
    text = re.sub(r'Dalam hal Anda menemukan inakurasi informasi yang termuat pada situs ini atau informasi yang seharusnya ada, namun belum tersedia, maka harap segera hubungi Kepaniteraan Mahkamah Agung RI melalui :\n', '', text)
    text = re.sub(r'Email : kepaniteraan@mahkamahagung\.go\.id\s+Telp : 021-384 3348 \(ext\.318\)\n', '', text)

    text = re.sub(r'\s+', ' ', text).strip()
    return text

# --- Fungsi untuk memuat cases.csv ---
def load_cases_data(processed_data_folder: str) -> pd.DataFrame:
    """
    Memuat file cases.csv yang sudah terstruktur dari Tahap 2.
    """
    cases_csv_path = os.path.join(processed_data_folder, 'cases.csv')
    print(f"Mencoba memuat cases.csv dari: {cases_csv_path}")
    if not os.path.exists(cases_csv_path):
        print(f"ERROR: File cases.csv TIDAK DITEMUKAN di '{cases_csv_path}'. Pastikan Tahap 2 sudah selesai dengan benar.")
        return pd.DataFrame()
    try:
        df = pd.read_csv(cases_csv_path)
        if df.empty:
            print(f"Peringatan: File cases.csv ditemukan tetapi KOSONG.")
        else:
            print(f"Berhasil memuat {len(df)} kasus dari cases.csv.")
            for col in ['judul', 'ringkasan_fakta', 'argumen_hukum_utama', 'text_pdf']:
                if col in df.columns:
                    df[col] = df[col].astype(str).fillna('')
        return df
    except Exception as e:
        print(f"ERROR: Gagal memuat cases.csv: {e}")
        return pd.DataFrame()

print("Fungsi utilitas Tahap 3 berhasil didefinisikan.")

Fungsi utilitas Tahap 3 berhasil didefinisikan.


In [24]:
import joblib # Import joblib untuk menyimpan model/vektor

# --- Konfigurasi Jalur Proyek ---
base_drive_path = '/content/drive/MyDrive/CBR_Data'
processed_data_folder = os.path.join(base_drive_path, 'data/processed')
eval_data_folder = create_path(os.path.join(base_drive_path, 'data/eval'))
# Tambahkan folder untuk menyimpan hasil model/vektor dari Tahap 3
models_output_folder = create_path(os.path.join(base_drive_path, 'models'))

# --- Muat Data Kasus ---
df_cases = load_cases_data(processed_data_folder)

# Inisialisasi variabel global yang akan digunakan di retrieve (dan Tahap 4)
vectorizer = None # Akan jadi TfidfVectorizer object jika TF-IDF
embedding_model = None # Akan jadi SentenceTransformer object jika BERT
model_for_retrieval_input = None # Vektor numpy array atau sparse matrix
vectorizer_type = None # String: "TF-IDF" atau "BERT_Embedding"

if df_cases.empty:
    print("Tidak ada data kasus yang dimuat. Proses representasi vektor tidak dapat dilanjutkan.")
else:
    documents = df_cases['text_pdf'].tolist()

    documents = [doc for doc in documents if doc.strip()]
    if not documents:
        print("ERROR: Tidak ada dokumen teks yang valid untuk dienkode. Pastikan kolom 'text_pdf' terisi.")
    else:
        print(f"Total dokumen teks valid untuk representasi: {len(documents)}")

        # --- PILIH SALAH SATU PENDEKATAN REPRESENTASI VEKTOR ---

        # =======================================================
        # PENDEKATAN 1: TF-IDF
        # =======================================================
        # print("\nMemilih pendekatan TF-IDF untuk representasi vektor...")
        # vectorizer = TfidfVectorizer(max_features=5000, stop_words=None)
        # case_vectors = vectorizer.fit_transform(documents)
        # print(f"Representasi TF-IDF selesai. Shape: {case_vectors.shape}")
        # vectorizer_type = "TF-IDF"
        # model_for_retrieval_input = case_vectors # Input untuk model ML

        # # --- SIMPAN OUTPUT TF-IDF ---
        # try:
        #     joblib.dump(vectorizer, os.path.join(models_output_folder, 'tfidf_vectorizer.joblib'))
        #     joblib.dump(model_for_retrieval_input, os.path.join(models_output_folder, 'tfidf_case_vectors.joblib'))
        #     with open(os.path.join(models_output_folder, 'vectorizer_type.txt'), 'w') as f:
        #         f.write(vectorizer_type)
        #     print(f"Output TF-IDF (vectorizer dan vectors) berhasil disimpan ke: {models_output_folder}")
        # except Exception as e:
        #     print(f"ERROR: Gagal menyimpan output TF-IDF: {e}")


        # =======================================================
        # PENDEKAN 2: BERT Embedding (Menggunakan Sentence Transformers)
        # =======================================================
        print("\nMemilih pendekatan BERT Embedding untuk representasi vektor...")
        try:
            print("Memuat model SentenceTransformer (paraphrase-multilingual-MiniLM-L12-v2)...")
            embedding_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

            batch_size = 32
            case_vectors_list = []
            for i in range(0, len(documents), batch_size):
                batch = documents[i:i+batch_size]
                embeddings = embedding_model.encode(batch, convert_to_tensor=True)
                case_vectors_list.append(embeddings)
                print(f"  Processed {i+len(batch)}/{len(documents)} documents for embedding.")

            case_vectors = torch.cat(case_vectors_list).cpu().numpy()
            print(f"Representasi BERT Embedding selesai. Shape: {case_vectors.shape}")
            vectorizer_type = "BERT_Embedding"
            model_for_retrieval_input = case_vectors # Input untuk model ML

            # --- SIMPAN OUTPUT BERT EMBEDDING ---
            try:
                np.save(os.path.join(models_output_folder, 'bert_case_vectors.npy'), model_for_retrieval_input)
                with open(os.path.join(models_output_folder, 'vectorizer_type.txt'), 'w') as f:
                    f.write(vectorizer_type)
                print(f"Output BERT Embedding (vectors) berhasil disimpan ke: {models_output_folder}")
            except Exception as e:
                print(f"ERROR: Gagal menyimpan output BERT Embedding: {e}")


        except Exception as e:
            print(f"ERROR: Gagal saat membuat BERT Embedding: {e}")
            print("Pastikan Anda memiliki RAM yang cukup di Colab (cek di bagian atas layar) atau coba model yang lebih kecil.")
            print("Anda mungkin perlu merestart runtime dan mencoba TF-IDF sebagai alternatif jika masalah berlanjut.")
            model_for_retrieval_input = None
            vectorizer_type = None

print("Representasi vektor selesai (atau dilewati karena error/data kosong).")

Folder '/content/drive/MyDrive/CBR_Data/data/eval' sudah ada di Google Drive.
Folder '/content/drive/MyDrive/CBR_Data/models' berhasil dibuat di Google Drive.
Mencoba memuat cases.csv dari: /content/drive/MyDrive/CBR_Data/data/processed/cases.csv
Berhasil memuat 65 kasus dari cases.csv.
Total dokumen teks valid untuk representasi: 65

Memilih pendekatan BERT Embedding untuk representasi vektor...
Memuat model SentenceTransformer (paraphrase-multilingual-MiniLM-L12-v2)...
  Processed 32/65 documents for embedding.
  Processed 64/65 documents for embedding.
  Processed 65/65 documents for embedding.
Representasi BERT Embedding selesai. Shape: (65, 384)
Output BERT Embedding (vectors) berhasil disimpan ke: /content/drive/MyDrive/CBR_Data/models
Representasi vektor selesai (atau dilewati karena error/data kosong).


In [25]:
# Pastikan df_cases, vectorizer_type, model_for_retrieval_input
# sudah didefinisikan dari sel sebelumnya.

# Inisialisasi variabel untuk Sel 5 (predict_outcome)
X_train, X_test, y_train, y_test = None, None, None, None
model_retrieval = None # Model ML (SVM/Naive Bayes) jika dilatih

if df_cases.empty or model_for_retrieval_input is None:
    print("Tidak dapat melanjutkan proses splitting data dan model retrieval karena data kosong atau representasi vektor gagal di Sel 3.")
else:
    print("\nMemulai proses Splitting Data dan Model Retrieval...")

    X = model_for_retrieval_input # Ini adalah representasi vektor dari dokumen

    valid_text_indices_mask = df_cases['text_pdf'].apply(lambda x: isinstance(x, str) and x.strip() != '')

    if valid_text_indices_mask.sum() == 0:
        print("Peringatan: Tidak ada dokumen teks yang valid untuk splitting. X kosong.")
    else:
        y = df_cases.loc[valid_text_indices_mask, 'klasifikasi'].fillna('UNKNOWN').astype(str)

        print(f"Jumlah sampel untuk splitting (X): {X.shape[0]}")
        print(f"Jumlah sampel untuk label (y): {y.shape[0]}")
        print(f"Jumlah kelas unik di y: {y.nunique()}")
        print(f"Distribusi kelas di y:\n{y.value_counts()}")

        can_stratify = False
        if y.nunique() > 1:
            class_counts = y.value_counts()
            if (class_counts >= 2).all(): # Minimal 2 sampel per kelas untuk stratify dengan test_size=0.2
                can_stratify = True
                print("Kondisi untuk stratify terpenuhi.")
            else:
                print("Peringatan: Beberapa kelas di 'y' memiliki kurang dari 2 sampel. Stratifikasi mungkin tidak bisa dilakukan atau akan menyebabkan error.")
                print(f"Kelas dengan kurang dari 2 sampel: {class_counts[class_counts < 2].index.tolist()}")
        else:
            print("Peringatan: Hanya ada satu kelas unik di 'y'. Stratifikasi tidak dapat dilakukan.")

        if can_stratify:
            try:
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
                print(f"Data berhasil dibagi dengan stratifikasi: Train={len(X_train)} samples, Test={len(X_test)} samples.")
                print("Distribusi kelas di y_train:\n", y_train.value_counts(normalize=True))
                print("Distribusi kelas di y_test:\n", y_test.value_counts(normalize=True))
            except ValueError as ve:
                print(f"ERROR: Gagal splitting dengan stratify=y: {ve}")
                print("Ini sering terjadi jika ada kelas dengan sampel sangat sedikit. Mencoba splitting tanpa stratifikasi.")
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
                print(f"Data berhasil dibagi TANPA stratifikasi: Train={len(X_train)} samples, Test={len(X_test)} samples.")
                print("Peringatan: Proporsi kelas mungkin tidak merata di train/test set.")
        else:
            print("Melakukan splitting tanpa stratifikasi karena kondisi tidak memungkinkan.")
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
            print(f"Data berhasil dibagi TANPA stratifikasi: Train={len(X_train)} samples, Test={len(X_test)} samples.")
            print("Peringatan: Proporsi kelas mungkin tidak merata di train/test set.")

        # --- Model Retrieval (Klasifikasi/Ranking) ---
        if X_train is not None and y_train is not None and len(y_train) > 0:
            if vectorizer_type == "TF-IDF":
                if y.nunique() > 1:
                    print("Melatih model Multinomial Naive Bayes (MultinomialNB) untuk klasifikasi/retrieval dengan TF-IDF...")
                    try:
                        model_retrieval = MultinomialNB()
                        model_retrieval.fit(X_train, y_train)
                        print("Model klasifikasi dilatih.")
                        # --- SIMPAN MODEL KLASIFIKASI ---
                        try:
                            joblib.dump(model_retrieval, os.path.join(models_output_folder, 'ml_classifier_model.joblib'))
                            print(f"Model klasifikasi berhasil disimpan ke: {models_output_folder}")
                        except Exception as e:
                            print(f"ERROR: Gagal menyimpan model klasifikasi: {e}")

                    except Exception as e:
                        print(f"ERROR: Gagal melatih MultinomialNB: {e}")
                        model_retrieval = None
                else:
                    print("Tidak cukup kelas unik di 'y' untuk melatih model klasifikasi TF-IDF.")
                    print("Retrieval akan dilakukan hanya berdasarkan kemiripan vektor secara langsung (Cosine Similarity).")

            elif vectorizer_type == "BERT_Embedding":
                print("Untuk BERT Embedding, retrieval biasanya dilakukan dengan Cosine Similarity langsung.")
                print("Tidak perlu melatih model klasifikasi baru di sini untuk retrieval kemiripan murni.")
                model_retrieval = None

            else:
                print("Tipe vectorizer tidak dikenal.")
                model_retrieval = None
        else:
            print("X_train atau y_train kosong. Tidak ada model retrieval yang dilatih.")
            model_retrieval = None

print("Splitting data dan setup model retrieval selesai.")


Memulai proses Splitting Data dan Model Retrieval...
Jumlah sampel untuk splitting (X): 65
Jumlah sampel untuk label (y): 65
Jumlah kelas unik di y: 6
Distribusi kelas di y:
klasifikasi
Pidana Khusus  Narkotika dan Psikotropika \n Pidana Khusus  Narkotika dan Psikotropika    51
Pidana Khusus  Narkotika dan Psikotropika                                                  6
Perdata Agama \n Perdata Agama  Perceraian                                                 5
Pidana Umum  Penghinaan                                                                    1
UNKNOWN                                                                                    1
Pidana Khusus  Pangan                                                                      1
Name: count, dtype: int64
Peringatan: Beberapa kelas di 'y' memiliki kurang dari 2 sampel. Stratifikasi mungkin tidak bisa dilakukan atau akan menyebabkan error.
Kelas dengan kurang dari 2 sampel: ['Pidana Umum  Penghinaan', 'UNKNOWN', 'Pidana Khusus  Pan

In [20]:
from typing import List

# Pastikan df_cases, vectorizer (jika TF-IDF), embedding_model (jika BERT),
# dan model_retrieval (jika ML classifier) sudah didefinisikan dari sel sebelumnya.

if df_cases.empty:
    print("Tidak dapat mendefinisikan fungsi retrieve karena data kasus kosong.")
else:
    def retrieve(query: str, k: int = 5) -> List[Dict[str, Any]]:
        """
        Menemukan top-k kasus lama yang paling mirip dengan query kasus baru.
        Args:
            query (str): Teks query kasus baru.
            k (int): Jumlah kasus teratas yang ingin dikembalikan.
        Returns:
            List[Dict[str, Any]]: Daftar top-k kasus terurut berdasarkan kemiripan,
                                  berisi case_id dan skor kemiripan.
        """
        print(f"\nMelakukan retrieval untuk query: '{query[:50]}...' (top-k={k})")

        # 1) Pre-process query (sesuai preprocessing dokumen)
        # Untuk kesederhanaan, gunakan pembersihan dasar yang sama dengan text_pdf
        processed_query = clean_text(query) # `clean_text` diasumsikan dari Tahap 1/2

        # 2) Hitung vektor query
        query_vector = None
        if vectorizer_type == "TF-IDF":
            # TF-IDF vectorizer harus sudah di-fit pada corpus dokumen
            if 'vectorizer' in locals() and vectorizer is not None:
                query_vector = vectorizer.transform([processed_query])
            else:
                print("Error: TF-IDF Vectorizer belum diinisialisasi.")
                return []
        elif vectorizer_type == "BERT_Embedding":
            if 'embedding_model' in locals() and embedding_model is not None:
                # Perlu memastikan embedding_model dapat diakses
                # Ini mengasumsikan model sudah dimuat di Sel 3
                query_vector = embedding_model.encode([processed_query], convert_to_tensor=True).cpu().numpy()
            else:
                print("Error: BERT Embedding Model belum diinisialisasi.")
                return []
        else:
            print("Error: Tipe vectorizer tidak dikenal.")
            return []

        if query_vector is None:
            return []

        results = []
        if vectorizer_type == "TF-IDF" and model_retrieval is not None:
            # Jika menggunakan model klasifikasi (SVM/Naive Bayes)
            print("Menggunakan model klasifikasi untuk memprediksi kategori query dan mencari kasus serupa di kategori itu.")
            # Ini adalah pendekatan klasifikasi, bukan retrieval kemiripan langsung di seluruh korpus
            # Untuk retrieval murni, kita biasanya mencari kemiripan vektor langsung.
            # Jika model_retrieval adalah classifier, maka ini akan memprediksi kelas
            # dan kita perlu mengambil kasus dari kelas yang diprediksi.

            # Prediksi probabilitas kelas untuk query
            if hasattr(model_retrieval, 'predict_proba'):
                proba = model_retrieval.predict_proba(query_vector)
                predicted_class_idx = np.argmax(proba)
                predicted_class = model_retrieval.classes_[predicted_class_idx]
                print(f"Query diprediksi masuk kategori: '{predicted_class}'")

                # Sekarang cari kasus yang paling mirip di antara kasus yang berlabel predicted_class
                # Ini memerlukan pemetaan kembali embeddings/TF-IDF ke dokumen asli dan labelnya.
                # Untuk kesederhanaan demo, kita akan kembali ke cosine similarity di seluruh korpus.
                # Implementasi klasifikasi untuk retrieval lebih kompleks dan butuh label jelas.

                # Fallback ke cosine similarity untuk retrieval kemiripan langsung
                similarities = cosine_similarity(query_vector, model_for_retrieval_input).flatten()
            else:
                print("Model retrieval tidak memiliki predict_proba. Melakukan retrieval berdasarkan cosine similarity langsung.")
                similarities = cosine_similarity(query_vector, model_for_retrieval_input).flatten()

        elif vectorizer_type == "BERT_Embedding" or (vectorizer_type == "TF-IDF" and model_retrieval is None):
            # 3) Hitung cosine-similarity dengan semua case vectors
            # Mengasumsikan model_for_retrieval_input sudah berisi embeddings/TF-IDF dari semua kasus
            similarities = cosine_similarity(query_vector, model_for_retrieval_input).flatten()

        else:
            print("Error: Konfigurasi model retrieval tidak didukung.")
            return []

        # Mendapatkan indeks kasus terurut berdasarkan kemiripan
        # Menggunakan argpartition untuk efisiensi jika k kecil
        top_k_indices = np.argpartition(similarities, -k)[-k:]
        # Urutkan berdasarkan kemiripan (descending)
        sorted_top_k_indices = top_k_indices[np.argsort(similarities[top_k_indices])][::-1]

        # 4) Kembalikan top-k case id (dan informasi relevan lainnya)
        # Pastikan indeks ini merujuk ke DataFrame asli
        # Karena `documents` dan `model_for_retrieval_input` mungkin tidak memiliki indeks yang sama
        # dengan df_cases akibat pembersihan dokumen kosong, kita perlu memetakan kembali.

        # Dapatkan indeks asli df_cases yang tidak dibuang karena text_pdf kosong
        original_valid_indices = df_cases[df_cases['text_pdf'].apply(lambda x: isinstance(x, str) and x.strip() != '')].index.tolist()

        # Petakan indeks dari `model_for_retrieval_input` kembali ke indeks DataFrame asli
        mapped_indices = [original_valid_indices[idx] for idx in sorted_top_k_indices]

        for idx in mapped_indices:
            case_info = df_cases.loc[idx]
            similarity_score = similarities[original_valid_indices.index(idx)] # Dapatkan skor kemiripan yang benar
            results.append({
                "case_id": case_info['nomor'], # Menggunakan nomor perkara sebagai case_id
                "judul": case_info['judul'],
                "klasifikasi": case_info['klasifikasi'],
                "similarity_score": float(similarity_score), # Pastikan float untuk JSON
                "link": case_info['link']
            })

        print(f"Retrieval selesai. Ditemukan {len(results)} kasus.")
        return results

print("Fungsi retrieval berhasil didefinisikan (atau dilewati).")

Fungsi retrieval berhasil didefinisikan (atau dilewati).


In [21]:
from typing import List

# Pastikan df_cases, vectorizer, embedding_model, model_for_retrieval_input,
# dan vectorizer_type sudah didefinisikan dari sel sebelumnya.
# Pastikan juga fungsi clean_text sudah ada (dari Sel 2).

if df_cases.empty:
    print("Tidak dapat mendefinisikan fungsi retrieve karena data kasus kosong.")
else:
    def retrieve(query: str, k: int = 5) -> List[Dict[str, Any]]:
        """
        Menemukan top-k kasus lama yang paling mirip dengan query kasus baru.
        Args:
            query (str): Teks query kasus baru.
            k (int): Jumlah kasus teratas yang ingin dikembalikan.
        Returns:
            List[Dict[str, Any]]: Daftar top-k kasus terurut berdasarkan kemiripan,
                                  berisi case_id dan skor kemiripan.
        """
        print(f"\nMelakukan retrieval untuk query: '{query[:50]}...' (top-k={k})")

        # 1) Pre-process query
        if 'clean_text' not in globals() or not callable(clean_text):
            print("ERROR: Fungsi 'clean_text' tidak ditemukan. Pastikan Sel 2 sudah dijalankan.")
            return []

        processed_query = clean_text(query)
        print(f"DEBUG: Query setelah pre-processing: '{processed_query[:50]}...'")

        # 2) Hitung vektor query
        query_vector = None
        if vectorizer_type == "TF-IDF":
            if vectorizer is not None:
                try:
                    query_vector = vectorizer.transform([processed_query])
                    print(f"DEBUG: Query vector TF-IDF shape: {query_vector.shape}")
                except Exception as e:
                    print(f"ERROR: Gagal transformasi query TF-IDF: {e}")
                    return []
            else:
                print("ERROR: TF-IDF Vectorizer belum diinisialisasi di Sel 3.")
                return []
        elif vectorizer_type == "BERT_Embedding":
            if embedding_model is not None:
                try:
                    query_vector = embedding_model.encode([processed_query], convert_to_tensor=True).cpu().numpy()
                    print(f"DEBUG: Query vector BERT Embedding shape: {query_vector.shape}")
                except Exception as e:
                    print(f"ERROR: Gagal membuat embedding query BERT: {e}")
                    return []
            else:
                print("ERROR: BERT Embedding Model belum diinisialisasi di Sel 3.")
                return []
        else:
            print("ERROR: Tipe vectorizer tidak dikenal atau belum diatur di Sel 3.")
            return []

        if query_vector is None or model_for_retrieval_input is None:
            print("ERROR: Query vector atau model_for_retrieval_input kosong. Tidak bisa menghitung kemiripan.")
            return []

        # Pastikan model_for_retrieval_input memiliki bentuk yang benar untuk cosine_similarity
        if not isinstance(model_for_retrieval_input, (np.ndarray, scipy.sparse.csr.csr_matrix, torch.Tensor)): # Tambahkan scipy.sparse jika pakai TF-IDF
             print(f"ERROR: model_for_retrieval_input memiliki tipe yang tidak didukung untuk similarity: {type(model_for_retrieval_input)}")
             return []

        # 3) Hitung cosine-similarity dengan semua case vectors
        try:
            similarities = cosine_similarity(query_vector, model_for_retrieval_input).flatten()
            print(f"DEBUG: Similarities shape: {similarities.shape}")
        except Exception as e:
            print(f"ERROR: Gagal menghitung cosine similarity: {e}")
            print(f"  Query vector shape: {query_vector.shape if query_vector is not None else 'None'}")
            print(f"  Corpus vectors shape: {model_for_retrieval_input.shape if model_for_retrieval_input is not None else 'None'}")
            return []

        # Mendapatkan indeks kasus terurut berdasarkan kemiripan
        if len(similarities) < k:
            print(f"Peringatan: Jumlah sampel ({len(similarities)}) kurang dari k ({k}). Mengembalikan semua sampel.")
            top_k_indices = np.arange(len(similarities))
        else:
            top_k_indices = np.argpartition(similarities, -k)[-k:]

        # Urutkan berdasarkan kemiripan (descending)
        sorted_top_k_indices = top_k_indices[np.argsort(similarities[top_k_indices])][::-1]

        # 4) Kembalikan top-k case id (dan informasi relevan lainnya)
        results = []
        # Dapatkan indeks asli df_cases yang tidak dibuang karena text_pdf kosong
        original_valid_indices = df_cases[df_cases['text_pdf'].apply(lambda x: isinstance(x, str) and x.strip() != '')].index.tolist()

        # Pastikan original_valid_indices memiliki panjang yang sama dengan model_for_retrieval_input.shape[0]
        if len(original_valid_indices) != model_for_retrieval_input.shape[0]:
            print(f"ERROR: Ketidakcocokan panjang antara original_valid_indices ({len(original_valid_indices)}) dan model_for_retrieval_input ({model_for_retrieval_input.shape[0]}).")
            print("Ini bisa terjadi jika ada dokumen yang dibuang setelah embedding atau kesalahan indexing.")
            return []

        for idx_in_corpus in sorted_top_k_indices:
            original_df_index = original_valid_indices[idx_in_corpus]
            case_info = df_cases.loc[original_df_index]
            similarity_score = similarities[idx_in_corpus] # Ambil skor dari array similarities
            results.append({
                "case_id": case_info['nomor'],
                "judul": case_info['judul'],
                "klasifikasi": case_info['klasifikasi'],
                "similarity_score": float(similarity_score),
                "link": case_info['link']
            })

        print(f"Retrieval selesai. Ditemukan {len(results)} kasus.")
        return results

print("Fungsi retrieval berhasil didefinisikan (atau dilewati).")

Fungsi retrieval berhasil didefinisikan (atau dilewati).


In [22]:
# --- Siapkan 5-10 query uji beserta ground-truth case_id (jika ada) ---
sample_queries = [
    {
        "query_id": "Q1_Narkotika_1",
        "text": "Seorang terdakwa ditangkap karena memiliki 5 gram sabu-sabu dan dijerat pasal 112 Undang-Undang Narkotika.",
        "ground_truth_case_id": [] # Ganti dengan nomor perkara yang benar dari data Anda jika ada
    },
    {
        "query_id": "Q2_Narkotika_2",
        "text": "Kasus peredaran gelap narkotika jenis ganja yang melibatkan lebih dari 1 kilogram.",
        "ground_truth_case_id": []
    },
    {
        "query_id": "Q3_Perdata_Wanprestasi_1",
        "text": "Gugatan wanprestasi karena salah satu pihak tidak memenuhi perjanjian jual beli tanah yang sudah disepakati.",
        "ground_truth_case_id": []
    },
    {
        "query_id": "Q4_Perdata_Perceraian_1",
        "text": "Permohonan cerai gugat di Pengadilan Agama karena perselisihan dan pertengkaran terus-menerus.",
        "ground_truth_case_id": []
    },
    {
        "query_id": "Q5_Pidana_Pencurian_1",
        "text": "Kasus pencurian sepeda motor yang dilakukan oleh anak di bawah umur di wilayah Jakarta.",
        "ground_truth_case_id": []
    }
]

# --- Lakukan pengujian awal retrieval ---
print("\n--- Melakukan Pengujian Awal Fungsi Retrieval ---")
retrieval_test_results = []
# Pastikan fungsi retrieve didefinisikan dan model_for_retrieval_input tidak kosong
if 'retrieve' in locals() and callable(retrieve) and model_for_retrieval_input is not None and model_for_retrieval_input.shape[0] > 0:
    for query_data in sample_queries:
        query_text = query_data['text']
        query_id = query_data['query_id']
        top_k_results = retrieve(query_text, k=5)
        print(f"\nQuery ID: {query_id}")
        print(f"Top 5 Kasus Mirip:")
        if top_k_results:
            for res in top_k_results:
                print(f"  - Case ID: {res['case_id']}, Judul: {res['judul'][:70]}..., Skor: {res['similarity_score']:.4f}")
        else:
            print("  Tidak ada hasil retrieval.")

        retrieval_test_results.append({
            "query_id": query_id,
            "query_text": query_text,
            "ground_truth_case_id": query_data['ground_truth_case_id'],
            "retrieved_cases": top_k_results
        })
else:
    print("Fungsi 'retrieve' tidak dapat ditemukan, atau model data kosong. Pastikan semua sel sebelumnya berhasil dijalankan dan data tersedia.")


# --- Simpan query uji ke /data/eval/queries.json ---
queries_json_path = os.path.join(eval_data_folder, 'queries.json')
try:
    with open(queries_json_path, 'w', encoding='utf-8') as f:
        json.dump(sample_queries, f, indent=4, ensure_ascii=False)
    print(f"\nQuery uji berhasil disimpan ke: {queries_json_path}")
except Exception as e:
    print(f"ERROR: Gagal menyimpan queries.json: {e}")

print("\nPROSES TAHAP 3 (CASE RETRIEVAL) SELESAI.")


--- Melakukan Pengujian Awal Fungsi Retrieval ---

Melakukan retrieval untuk query: 'Seorang terdakwa ditangkap karena memiliki 5 gram ...' (top-k=5)
DEBUG: Query setelah pre-processing: 'Seorang terdakwa ditangkap karena memiliki 5 gram ...'
DEBUG: Query vector BERT Embedding shape: (1, 384)
DEBUG: Similarities shape: (65,)
Retrieval selesai. Ditemukan 5 kasus.

Query ID: Q1_Narkotika_1
Top 5 Kasus Mirip:
  - Case ID: 83/Pid.Sus/2015/PN. Nnk, Judul: Putusan PN NUNUKAN Nomor 83/Pid.Sus/2015/PN. Nnk Tanggal 6 Juli 2105 —..., Skor: 0.2941
  - Case ID: 326/PID.SUS/2025/PT PBR, Judul: Putusan PT PEKANBARU Nomor 326/PID.SUS/2025/PT PBR Tanggal 26 Juni 202..., Skor: 0.2934
  - Case ID: 326/PID.SUS/2025/PT PBR, Judul: Putusan PT PEKANBARU Nomor 326/PID.SUS/2025/PT PBR Tanggal 26 Juni 202..., Skor: 0.2934
  - Case ID: 5492 K/Pid.Sus/2024, Judul: Putusan MAHKAMAH AGUNG Nomor 5492 K/Pid.Sus/2024 Tanggal 19 September ..., Skor: 0.2928
  - Case ID: 63/Pid.Sus/2025/PN Ksp, Judul: Putusan PN KUALA 

/tmp/ipython-input-21-4261560080.py:63: DeprecationWarning: Please import `csr_matrix` from the `scipy.sparse` namespace; the `scipy.sparse.csr` namespace is deprecated and will be removed in SciPy 2.0.0.
  if not isinstance(model_for_retrieval_input, (np.ndarray, scipy.sparse.csr.csr_matrix, torch.Tensor)): # Tambahkan scipy.sparse jika pakai TF-IDF
/tmp/ipython-input-21-4261560080.py:63: DeprecationWarning: Please import `csr_matrix` from the `scipy.sparse` namespace; the `scipy.sparse.csr` namespace is deprecated and will be removed in SciPy 2.0.0.
  if not isinstance(model_for_retrieval_input, (np.ndarray, scipy.sparse.csr.csr_matrix, torch.Tensor)): # Tambahkan scipy.sparse jika pakai TF-IDF
/tmp/ipython-input-21-4261560080.py:63: DeprecationWarning: Please import `csr_matrix` from the `scipy.sparse` namespace; the `scipy.sparse.csr` namespace is deprecated and will be removed in SciPy 2.0.0.
  if not isinstance(model_for_retrieval_input, (np.ndarray, scipy.sparse.csr.csr_matrix, 

DEBUG: Query vector BERT Embedding shape: (1, 384)
DEBUG: Similarities shape: (65,)
Retrieval selesai. Ditemukan 5 kasus.

Query ID: Q4_Perdata_Perceraian_1
Top 5 Kasus Mirip:
  - Case ID: 1489/Pdt.G/2025/PA.Pwd, Judul: Putusan PA PURWODADI Nomor 1489/Pdt.G/2025/PA.Pwd Tanggal 19 Juni 2025..., Skor: 0.4230
  - Case ID: 154/PID.SUS/2025/PT MAM, Judul: Putusan PT SULAWESI BARAT Nomor 154/PID.SUS/2025/PT MAM Tanggal 26 Jun..., Skor: 0.3536
  - Case ID: 1445/Pdt.G/2025/PA.Pwd, Judul: Putusan PA PURWODADI Nomor 1445/Pdt.G/2025/PA.Pwd Tanggal 19 Juni 2025..., Skor: 0.3398
  - Case ID: 1445/Pdt.G/2025/PA.Pwd, Judul: Putusan PA PURWODADI Nomor 1445/Pdt.G/2025/PA.Pwd Tanggal 19 Juni 2025..., Skor: 0.3398
  - Case ID: 106/Pid.Sus/2025/PN Pkb, Judul: Putusan PN Pangkalan Balai Nomor 106/Pid.Sus/2025/PN Pkb Tanggal 26 Ju..., Skor: 0.3387

Melakukan retrieval untuk query: 'Kasus pencurian sepeda motor yang dilakukan oleh a...' (top-k=5)
DEBUG: Query setelah pre-processing: 'Kasus pencurian sepeda m

/tmp/ipython-input-21-4261560080.py:63: DeprecationWarning: Please import `csr_matrix` from the `scipy.sparse` namespace; the `scipy.sparse.csr` namespace is deprecated and will be removed in SciPy 2.0.0.
  if not isinstance(model_for_retrieval_input, (np.ndarray, scipy.sparse.csr.csr_matrix, torch.Tensor)): # Tambahkan scipy.sparse jika pakai TF-IDF
